In [1]:
import os
os.getcwd()
os.chdir('./tests rpi4/scans')

In [2]:
import glob
docLogs = sorted(glob.glob(os.path.join("log", "*", "doc.log")))

def docTiming(fn):
    timing = []
    with open(fn) as fd:
        timing = [l for l in fd.readlines() if l.strip().startswith("Processed")]
    if len(timing):
        timing = timing[0].split()
        timing = int(timing[1]), int(timing[-1].rstrip('s.'))
    return timing

single, multi = [], [] # seconds per page in single or multi sheet mode
for fn in docLogs:
    timing = docTiming(fn)
    if len(timing) < 2:
        continue
    npages, secs = timing
    if npages > 2:
        multi += (secs/npages,)
    else:
        single += (secs/npages,)
    #print(npages, secs)
print("m", multi)
print("s", single)

m [258.375, 274.3333333333333, 273.25, 323.9166666666667, 311.125, 299.3333333333333, 289.5, 293.0833333333333]
s [243.0, 322.5, 245.0, 277.5, 241.5, 292.5, 241.5, 248.0, 318.5, 244.5]


In [3]:
import numpy as np
multi = np.array(multi)
single = np.array(single)
import pandas as pd
arrs    = (single, multi)
indices = ('single sheet (2 pages)', 'multiple sheets')
lst = []
for arr in arrs:
    lst.append({'median (secs)': np.median(arr), 'mean (secs)': arr.mean(),
                '\xb1 (secs)': arr.std(), '#runs': len(arr)})
df = pd.DataFrame(lst, index=indices, columns=lst[0].keys())
df

,median (secs),mean (secs),± (secs),#runs
single sheet (2 pages),246.5,267.4,31.2,10
multiple sheets,291.3,290.4,20.1,8


In [4]:
from tabulate import tabulate
print(tabulate(df, tablefmt="pipe", headers="keys", floatfmt='1.0f'))

|                        |   median (secs) |   mean (secs) |   ± (secs) |   #runs |
|:-----------------------|----------------:|--------------:|-----------:|--------:|
| single sheet (2 pages) |             246 |           267 |         31 |      10 |
| multiple sheets        |             291 |           290 |         20 |       8 |
